# To visualize on Qgis

In [ ]:
import pandas as pd
import numpy as np
import os

In [1]:
# importing necessary functions from other files

import os
import sys

# to facilitate the use of notebooks
%load_ext autoreload
%autoreload 2

# Get the current directory of the notebook
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
functions_dir = os.path.join(parent_dir, "common")
print(functions_dir)

# Add this directory to sys.path
if functions_dir not in sys.path:
    sys.path.append(functions_dir)

from merge_metadata import add_HS, add_coord

/Users/mahlia/Desktop/SeineProject/python_codes/common


In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/Users/mahlia/Desktop/SeineProject/python_functions")
from merge_metadata import add_HS, add_coord

In [ ]:
file_names = os.listdir(os.path.join(os.getcwd(),"phychem_param_80_19", "all_param"))

for i, file in enumerate(file_names):
    csv = file.split(".")[1]
    if csv != "csv":
        file_names.pop(i)
file_names

In [ ]:
# fill with othere files
french_name_param = []
code_param = []
list_df_files = []

for i, file_name in enumerate(file_names):
    print(file_name)
    df = pd.read_csv(os.path.join("phychem_param_80_19","all_param",file_name),  delimiter=";") # correctly chose del
    df["date_prelevement"] = pd.to_datetime(df["date_prelevement"])
    df = df.sort_values("date_prelevement")

    french_name_param += df["libelle_parametre"].unique().tolist()
    print(df["libelle_parametre"].unique().tolist())
    code_param += df["code_parametre"].unique().tolist()

    list_df_files.append(df)
    print("df exported")

In [ ]:
df_files = pd.concat(list_df_files).reset_index(drop=True)

date_limite = pd.Timestamp('1980-03-01')
df_files = df_files.loc[df_files['date_prelevement'] >= date_limite].reset_index(drop=True) # only consider from sept bc start of hydrological year in france

In [ ]:
# chose this

year = 1998
season = "SP"
param = "Ammonium"

season_dict = {
    1 : "WT",
    2 : "WT",
    3 : "SP",
    4 : "SP",
    5 : "SP",
    6 : "SM",
    7 : "SM",
    8 : "SM",
    9 : "FL",
    10 : "FL",
    11 : "FL",
    12 : "WT"
}

In [ ]:
# get months from given season
months = [k for k, v in season_dict.items() if v == 'SP']

# dates in correct format
start_date = pd.Timestamp(f'{year}-{months[0]}-01')
end_date = pd.Timestamp(f'{year}-{months[-1]+1}-01') # end_date is first day of next season

# filter only between correct dates
df_visu = df_files.loc[(df_files['date_prelevement'] >= start_date) & (df_files['date_prelevement'] <= end_date)] 
df_visu = df_visu.loc[df_visu['libelle_parametre'] == param].reset_index(drop=True)


In [ ]:
# group to calculate median
df_visu_grouped = df_visu.groupby("code_station").agg({
    "date_prelevement" : list,
    "resultat" : list
}).reset_index()
df_visu_grouped.head()

# calculate median
df_visu_grouped.resultat = df_visu_grouped.resultat.apply(lambda x: np.median(np.array(x)) if isinstance(x, list) else x)

# add coordinates from metadata file
df_visu_grouped = add_coord(type="Physico-chemical", left_df=df_visu_grouped, left_id="code_station")

In [ ]:
# check if min and max make sense
df_visu_grouped.resultat.max(), df_visu_grouped.resultat.min()

In [ ]:
# change param name if many words
# if param == "Azote Kendjhal": name_visu = f"visu_nitroKendj_{season}_{year}.csv"

# create csv to visualize on Qgis

name_visu = f"visu_{param}_{season}_{year}.csv"
df_visu_grouped.to_csv(name_visu, sep=';', index=False)